# Translate eisenscript to eisenxml using lark

In [1]:
from pathlib import Path
from lark import Lark
from lark.visitors import Interpreter, Transformer, v_args
parser = Lark.open('eisenscript_02.lark')

In eisen tree structure I have nodes for
 - main
 - global_md
 - entry
 - call
 - loop
 - trans 
 - w_mod
 - md_mod

 will need an Interpreter method for each which adds something to the xml tree  
 how do I get token values?
 look   @visit_children_decor

In [2]:
# minimal example
test = """ 2 * { x 1 y 2} seed
{ rx 1 ry 2} seed
rule seed  { 
  box 
}
"""

In [3]:
ball = """set maxdepth 2000
R1 

rule R1 w 10 { 
{ x 1  rz 3 ry 5  } R1
{ s 1 1 0.1 sat 0.9 } box
} 
"""

In [4]:
small = parser.parse(test)
small

Tree(Token('RULE', 'main'), [Tree(Token('RULE', 'entry'), [Tree(Token('RULE', 'call'), [Tree(Token('RULE', 'loop'), [Tree('repeat', [Token('NUMBER', '2')]), Tree(Token('RULE', 'transgroup'), [Tree(Token('RULE', 'trans'), [Token('TID', 'x'), Token('VALUE', '1')]), Tree(Token('RULE', 'trans'), [Token('TID', 'y'), Token('VALUE', '2')])])]), Tree('rulename', [Token('RULENAME', 'seed')])]), Tree(Token('RULE', 'call'), [Tree(Token('RULE', 'loop'), [Tree(Token('RULE', 'transgroup'), [Tree(Token('RULE', 'trans'), [Token('TID', 'rx'), Token('VALUE', '1')]), Tree(Token('RULE', 'trans'), [Token('TID', 'ry'), Token('VALUE', '2')])])]), Tree('rulename', [Token('RULENAME', 'seed')])])]), Tree(Token('RULE', 'ruledef'), [Tree('rulename', [Token('RULENAME', 'seed')]), Tree(Token('RULE', 'call'), [Tree(Token('RULE', 'loop'), []), Tree('rulename', [Token('RULENAME', 'box')])])])])

In [5]:
print(small.pretty())

main
  entry
    call
      loop
        repeat	2
        transgroup
          trans
            x
            1
          trans
            y
            2
      rulename	seed
    call
      loop
        transgroup
          trans
            rx
            1
          trans
            ry
            2
      rulename	seed
  ruledef
    rulename	seed
    call
      loop
      rulename	box



In [6]:
from xml.etree import ElementTree as et
from xml.dom import minidom


In [139]:
def prettify(elem):
    """Return a pretty-printed XML string for the Element."""
    rough_string = et.tostring(elem, "utf-8")
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="  ")


class Itest(Interpreter):
    def convert(self, tree):
        self.xmltree = et.Element("rules")
        self.active = self.xmltree
        self._visit_tree(tree)
        return prettify(self.xmltree)

    def main(self, tree):
        self.visit_children(tree)

    def entry(self, tree):
        self.active = et.SubElement(self.xmltree, "rule")
        self.active.set("name", "entry")
        self.visit_children(tree)
        self.active = self.xmltree

    def ruledef(self, tree):
        self.active = et.SubElement(self.xmltree, "rule")
        current = self.active
        data = self.visit_children(tree)
        current.set("name", data[0])
        self.active = self.xmltree

    def call(self, tree):
        """
        most of the logic and xml generation happens here
        data has structure
        multiple loops
        data = [{'count': '10', 'tgroup': 'tx 1', 'rname': 'r1_00'},
                {'count': '20', 'tgroup': 'ty 2', 'rname': 'r1_01'},
                {'count': '5', 'tgroup': 'tz 3', 'rname': 'r1'}]
        single loop
        data = [{'count': '1', 'tgroup': 'tx 1', 'rname': 'r1}]
        """
        data = self.visit_children(tree)
        rname = data[-1]
        data = data[:-1]
        # make the last loop rname and create names for any previous extra loops
        data[-1]["rname"] = rname
        for i, loop_dict in enumerate(data[:-1]):
            loop_dict["rname"] = f"{rname}_{i:02}"

        for loop in data:
            print(loop)
        print()
            
        # the first loop is a call within the current rule
        if rname in ["box", "grid", "sphere", "line"]:
            call = et.SubElement(self.active, "instance")
            call.set("shape", data[0]["rname"])
        else:
            call = et.SubElement(self.active, "call")
            call.set("rule", data[0]["rname"])

        call.set("count", data[0]["count"])
        call.set("transforms", data[0]["tgroup"])

        # subsequent loops are converted to rules with a single call to the next rule
        for i, loop_dict in enumerate(data[1:]):
            self.active = et.SubElement(self.xmltree, "rule")
            self.active.set("name", data[i]['rname'])
            call = et.SubElement(self.active, "call")
            call.set("rule", loop_dict["rname"])
            call.set("count", loop_dict["count"])
            call.set("transforms", loop_dict["tgroup"])

    def transgroup(self, tree):
        # joins all the transforms inside {} into a string
        data = " ".join(self.visit_children(tree))
        return data

    @v_args(meta=True)
    def trans(self, kids, meta):
        # called for each transform, returns a string
        tid = meta[0]
        values = meta[1:]

        if tid == "s" and len(values) == 1:
            tid = "sa"
        if tid in ("x", "y", "z"):
            tid = "t" + tid
        if tid in ("tx", "ty", "tz", "rx", "ry", "rz", "sa", "s"):
            tstr = tid + " " + " ".join(values)
        else:
            tstr = ""
        return tstr

    def loop(self, tree):
        # an optional count and the list of transforms in {} brackets
        # creates a dict containing count and transform group
        data = self.visit_children(tree)

        if len(data) == 0:
            loop_dict = {"count": "0", "tgroup": ""}
        if len(data) == 1:
            loop_dict = {"count": "1", "tgroup": data[0]}
        if len(data) == 2:
            loop_dict = {"count": data[0], "tgroup": data[1]}

        return loop_dict

    @v_args(inline=True)
    def rulename(self, rname):
        return str(rname)

    @v_args(inline=True)
    def repeat(self, c):
        return str(c)

    @v_args(inline=True)
    def wmod(self, weight):
        self.active.set("weight", weight)

    @v_args(meta=True)
    def mmod(self, kids, meta):
        self.active.set("max_depth", meta[0])
        if len(meta) > 1:
            self.active.set("successor", meta[1])


In [140]:
# minimal example
test = """ 2 * { x 1 y 2} seed
{ rx 1 ry 2} seed
rule seed  { 
  box 
}
"""
test = parser.parse(test)
print(Itest().convert(test))

{'count': '2', 'tgroup': 'tx 1 ty 2', 'rname': 'seed'}

{'count': '1', 'tgroup': 'rx 1 ry 2', 'rname': 'seed'}

{'count': '0', 'tgroup': '', 'rname': 'box'}

<?xml version="1.0" ?>
<rules>
  <rule name="entry">
    <call rule="seed" count="2" transforms="tx 1 ty 2"/>
    <call rule="seed" count="1" transforms="rx 1 ry 2"/>
  </rule>
  <rule name="seed">
    <instance shape="box" count="0" transforms=""/>
  </rule>
</rules>




why do I get   
`loop  [Token('NUMBER', '2'), 'tx 1', 'ty 2']`
instead of 
`loop  ['2', 'tx 1', 'ty 2']`
as repeat returns 2, change to `return int(c)`

best structure for data returned from call's kids might be
`[2, 'tx 1 ty 2', 'seed' ]`

as of now have
`[[2, 'tx 1', 'ty 2'], 'seed']`
or 
`[['rx 1', 'ry 2'], 'seed']`
and for nested loops
`[[10, 'tx 1'], [20, 'ty 2'], 'r1']`
which will be hard to separate optional count.

now have
`call  [[2, 'tx 1 ty 2'], 'seed']`
`call  [['rx 1 ry 2'], 'seed']`
`call [[10, 'tx 1'], [20, 'ty 2'], 'r1']`
`call  [[], 'sphere']`
which will work


In [143]:
ball = """set maxdepth 2000
R1 

rule R1 w 10 md 20 > R2  { 
{ x 1  } R1
{ s 1 1 0.1 } box
} 
rule R2  { 
sphere
} 
"""
balltree = parser.parse(ball)
print(Itest().convert(balltree))

{'count': '0', 'tgroup': '', 'rname': 'R1'}

{'count': '1', 'tgroup': 'tx 1', 'rname': 'R1'}

{'count': '1', 'tgroup': 's 1 1 0.1', 'rname': 'box'}

{'count': '0', 'tgroup': '', 'rname': 'sphere'}

<?xml version="1.0" ?>
<rules>
  <rule name="entry">
    <call rule="R1" count="0" transforms=""/>
  </rule>
  <rule weight="10" max_depth="20" successor="R2" name="R1">
    <call rule="R1" count="1" transforms="tx 1"/>
    <instance shape="box" count="1" transforms="s 1 1 0.1"/>
  </rule>
  <rule name="R2">
    <instance shape="sphere" count="0" transforms=""/>
  </rule>
</rules>



can we make colour a generic string  
"red" 
           | "black"
           | "white"
           | "blue"
           | "green"

oct below has nested rules and currently fails to parse
change   
`call: count* loop* rn`  
to  
`call: (count* loop*)* rn`  

This only gets the last count * loop.  

My old code did this:

```xml
<?xml version="1.0" ?>
<rules max_depth="100">
  <rule name="entry">
    <call transforms="ry 36" count="10" rule="r1_00"/>
  </rule>
  <rule name="r1_00">
    <call transforms="ry 10" count="30" rule="r1"/>
  </rule>
  <rule name="r1" weight="20">
    <instance transforms="s 1 0.2 0.5" shape="box"/>
    <call transforms="sa 0.9 rz 5 rx 5 tx 1" rule="r1"/>
  </rule>
  <rule name="r1" weight="20">
    <instance transforms="s 1 0.2 0.5" shape="box"/>
    <call transforms="sa 0.99 rz -5 rx -5 tx 1" rule="r1"/>
  </rule>
</rules>
```

It's created an extra rule "r1_00" and dropped the third nested rule. The code does say it only deals with two loops per call.

Lark version visits all the loops but keeps overriding the atrributes
Creating the extra rules here could be done by adding to the tree?

Logic:  
if more than one loop per call:
  

item? - Zero or one instances of item (”maybe”)
item* - Zero or more instances of item
item+ - One or more instances of item
item ~ n - Exactly n instances of item
item ~ n..m - Between n to m instances of item (not recommended for wide ranges, due to performance issues)


In [142]:
oct2 = """set maxdepth 100

10 * { x 1} 20 * { y 2 }  r1

rule r1  {
    2 * {rx 1} box
    sphere
}"""

t = parser.parse(oct2)
print(Itest().convert(t))

"""
<?xml version="1.0" ?>
<rules max_depth="100">
  <rule name="entry">
    <call transforms="tx 1" count="10" rule="r1_00"/>
  </rule>
  <rule name="r1_00">
    <call transforms="ty 2" count="20" rule="r1"/>
  </rule>
  <rule name="r1">
    <instance transforms="" shape="box"/>
  </rule>
</rules>
"""

{'count': '10', 'tgroup': 'tx 1', 'rname': 'r1_00'}
{'count': '20', 'tgroup': 'ty 2', 'rname': 'r1'}

{'count': '2', 'tgroup': 'rx 1', 'rname': 'box'}

{'count': '0', 'tgroup': '', 'rname': 'sphere'}

<?xml version="1.0" ?>
<rules>
  <rule name="entry">
    <call rule="r1_00" count="10" transforms="tx 1"/>
  </rule>
  <rule name="r1_00">
    <call rule="r1" count="20" transforms="ty 2"/>
  </rule>
  <rule name="r1">
    <instance shape="box" count="2" transforms="rx 1"/>
    <instance shape="sphere" count="0" transforms=""/>
  </rule>
</rules>



'\n<?xml version="1.0" ?>\n<rules max_depth="100">\n  <rule name="entry">\n    <call transforms="tx 1" count="10" rule="r1_00"/>\n  </rule>\n  <rule name="r1_00">\n    <call transforms="ty 2" count="20" rule="r1"/>\n  </rule>\n  <rule name="r1">\n    <instance transforms="" shape="box"/>\n  </rule>\n</rules>\n'

In [12]:
print(t.pretty())

main
  global_md	100
  entry
    call
      loop
        repeat	10
        transgroup
          trans
            x
            1
      loop
        repeat	20
      loop
        transgroup
          trans
            y
            2
      rulename	r1
  ruledef
    rulename	r1
    call
      loop
        transgroup
          trans
            rx
            1
      rulename	box
    call
      loop
      rulename	sphere



Logic for two loops:  
if 2 loops on call:  
    make another rule from 2nd loop  
    and call it from 1st loop  

logic for more loops  
    make a rule for each extra loop  
    call them in  a chain  

have in eisenscript_02.lark pushed the repeat down to the loop level 

All the bits for a new rule in one place.
Who should handle its creation? call or loop?  


I think I have this as above, handling any nmber of multiple rules  
Not dealing with set directives yet.

In [13]:
x = ['a']

In [14]:
len(x) == True

True